In [1]:
import pandas as pd 
import numpy as np

In [2]:
data_train=pd.read_csv('data_train.txt',index_col=[0],header=0)
# data_valid=pd.read_csv('data_valid.txt',index_col=[0],header=0)
data_valid=pd.read_csv('data_predict.txt',index_col=[0],header=0)

C:\Users\oyrx\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def is_noise(data_train,data_test):
    #data_train 需要标记出noise的数据，以便建模
    data_train['is_noise']=0
    data_train['is_noise'][data_train['non_ch']==1]=1
    data_train['is_noise'][data_train['length_all']<=5]=1
    data_train['is_noise'][data_train['auto']==1]=1
    data_train['is_noise'][data_train['interaction']==1]=1
    data_train['is_noise'][data_train['vote']==1]=1
    data_train['is_noise'][data_train['ad']==1]=1
    
    #data_test 需要标记出noise的数据，以便预测
    data_test['is_noise']=0
    data_test['is_noise'][data_test['non_ch']==1]=1
    data_test['is_noise'][data_test['length_all']<=5]=1
    data_test['is_noise'][data_test['auto']==1]=1
    data_test['is_noise'][data_test['interaction']==1]=1
    data_test['is_noise'][data_test['vote']==1]=1
    data_test['is_noise'][data_test['ad']==1]=1
    
    #data_test的用户数据需要从data_train中获取
    user_history=data_train[['user_id','number_in_train','forward_max','comment_max','like_max','forward_min','comment_min','like_min','forward_mean','comment_mean','like_mean','forward_more_ave_pr','comment_more_ave_pr','like_more_ave_pr','max_f/l','max_c/l','min_f/l','min_c/l','mean_f/l','mean_c/l']]
    user_history=user_history.drop_duplicates()
    data_test=pd.merge(data_test,user_history,on='user_id',how='left',indicator=False)
    data_test=data_test.fillna(-1)  #将缺失用户的数值标记为-1，为了与互动数为0的用户区分开来
    #如果data_test中出现新用户，标记为noise值
    data_test['is_noise'][data_test['number_in_train']==-1]=1
    #如果data_test中的旧用户的互动数为0，标记为noise值
    data_test['is_noise'][(data_test['forward_max']+data_test['comment_max']+data_test['like_max'])==0]=1
    return data_train,data_test

In [4]:
data_train,data_valid=is_noise(data_train,data_valid)

C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher

In [5]:
#data_valid[['text','repost','comments','likes']][data_valid['is_noise']==1]

In [6]:
#data_train[['text','repost','comments','likes']][data_train['is_noise']==1]

In [7]:
#data_train.to_csv('data_train.txt',header=True)
#data_valid.to_csv('data_valid.txt',header=True)
data_valid.to_csv('data_predict.txt',header=True)

In [8]:
data2=pd.read_csv('data_valid.txt',index_col=[0],header=0)
data2.head()

,user_id,weibo_id,time,repost,comments,likes,tfidf,text,time_weekday,time_weekend,...,like_mean,forward_more_ave_pr,comment_more_ave_pr,like_more_ave_pr,max_f/l,max_c/l,min_f/l,min_c/l,mean_f/l,mean_c/l
0,f349a67d1cd7c8683c5bbc5f8486e193,83674a60e5310195fc35d97ea8f45c46,2015-07-15 01:16:24,0,0,0,1.390068,论优衣库试衣间隔音效果好坏？ http://t.cn/RL5aSzp（分享自 @知乎）,3,0,...,0.250000,0.011364,0.125000,0.000000,2.000000,9.500000,0.0,0.0,0.500000,3.272727
1,875a4a77b339d93f819e2c4de5bd0b57,f2cdcdbcec9ff47cbb3c6a636e4b92a3,2015-07-01 04:11:48,0,0,0,0.731223,#IT#【武汉一专车司机因毒驾被治安拘留】新华网武汉6月29日电(记者冯国栋)记者28日从武...,3,0,...,0.009554,0.000074,0.000112,0.000223,1.100000,0.200000,0.0,0.0,1.112840,0.377432
2,0fc17bf5e2dc789dd48505df1f5b14fd,4c1e2418127811d212d0e3867a99db3e,2015-07-13 05:07:28,0,0,1,2.988692,羽田机场~~ http://t.cn/RLGJidL,1,0,...,0.047619,0.000000,0.142857,0.000000,0.000000,2.000000,0.0,0.0,0.000000,7.000000
3,dd749a5af07c04ce7de451273a983671,419dd71d562883ef836e774bc3f4e163,2015-07-30 14:24:28,0,0,0,0.962645,"分享了-KUTLUK-的歌单《Piano,奏响灵魂深处...》 http://t.cn/...",4,0,...,0.194737,0.005263,0.126316,0.021053,1.333333,2.333333,0.0,0.0,0.270270,2.081081
4,6623347e5f19f35f2d02ad515b96524c,9a2f48a870843d1964a03c6642b309d5,2015-07-21 01:06:53,0,0,0,1.018777,#糟蹋酒鬼菜#老板每次推荐完他们家的下酒菜，就会拼命的安利上等的酒……我每次都会妥协的喝两罐汽水。,2,0,...,0.210634,0.022495,0.114519,0.010225,5.750000,2.750000,0.0,0.0,2.038835,2.067961


In [10]:
data2.columns

Index(['user_id', 'weibo_id', 'time', 'repost', 'comments', 'likes', 'tfidf',
       'text', 'time_weekday', 'time_weekend', 'time_hour', 'panduan',
       'length_all', 'length_chinese', 'english', 'non_ch', 'sharing', 'auto',
       'interaction', 'book', 'mention', 'vote', 'lottery', 'emoji', 'video',
       'http', 'app', 'title', 'ad', 'hotwords', 'keywords', 'is_noise',
       'number_in_train', 'forward_max', 'comment_max', 'like_max',
       'forward_min', 'comment_min', 'like_min', 'forward_mean',
       'comment_mean', 'like_mean', 'forward_more_ave_pr',
       'comment_more_ave_pr', 'like_more_ave_pr', 'max_f/l', 'max_c/l',
       'min_f/l', 'min_c/l', 'mean_f/l', 'mean_c/l'],
      dtype='object')